## Defino la función kFold

### Parámetros

**Datos:** Path al archivos train.csv.<br>
**cantFolds:** Es la cantidad de folds. Se ejecuta kNN por cada fold, así que no conviene poner más de 5 para que no demore tanto la ejecución.<br>
**k:** Parámetro k de kNN.<br>
**alfa:** Parámetro alfa de PCA. Si ponen alfa=0 se ejecuta kNN sin PCA.<br>
**semilla:** Es un parámetro opcional para que los resultados sean reproducibles, porque los datos de entrada se distribuyen en los folds de manera aleatoria.

Devuelve una lista con el accuracy de cada ejecución de kNN

In [8]:
import pandas as pd
import numpy as np
import metnum
from sklearn.metrics import accuracy_score

def kFold(datos, cantFolds, k, alfa, semilla=None):
    # Cargo los datos
    df_train = pd.read_csv(datos)
    
    # Desordeno los datos
    df_train = df_train.sample(frac=1, random_state=semilla)
    
    # Separo los datos en pixeles y etiquetas y los guardo en arrays de numpy
    X = df_train[df_train.columns[1:]].values
    y = df_train["label"].values.reshape(-1, 1)
    
    # Aplico PCA
    if (alfa > 0):
        pca = metnum.PCA(alfa)
        X = pca.transform(X)
    
    # Particiono el conjunto de datos en k folds del mismo tamaño
    folds = []
    limites = [0]
    
    for i in range(1,cantFolds+1):
        datosPorFold = len(df_train)//cantFolds
        limites.append(i*datosPorFold)
    
    for i in range(len(limites)-1):
        folds.append([X[limites[i]:limites[i+1]], y[limites[i]:limites[i+1]]])
        
    # Aplico kNN
    clf = metnum.KNNClassifier(k)
    acc = []
    
    for i in range(cantFolds):
        # Creo una lista para separar el índice del fold que voy a usar para validar
        indices = [[],i]
        for j in range(cantFolds):
            if (j != i):
                indices[0].append(j)

        X_val = folds[indices[1]][0]
        y_val = folds[indices[1]][1]

        # Uno todos los folds que voy a usar para entrenar en un único array de numpy
        X_train = folds[indices[0][0]][0]
        y_train = folds[indices[0][0]][1]
        indices[0].pop()

        for j in indices[0]:
            X_train = np.concatenate([X_train, folds[j][0]])
            y_train = np.concatenate([y_train, folds[j][1]])

        # Aplico kNN
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        acc.append(accuracy_score(y_val, y_pred))
    
    return acc

## Pruebo la función

### Primero sin semilla

In [11]:
%%time
resultados1 = kFold("../data/train.csv",5,10,20)

CPU times: user 5min 40s, sys: 2.23 s, total: 5min 43s
Wall time: 5min 42s


In [12]:
resultados1

[0.9651190476190477,
 0.9602380952380952,
 0.9589285714285715,
 0.9629761904761904,
 0.9607142857142857]

### Ahora especifico una semilla para ver si consigo resultados reproducibles

In [14]:
semilla = 1000

resultados2 = kFold("../data/train.csv",5,10,20,semilla)

In [15]:
resultados2

[0.9602380952380952,
 0.9603571428571429,
 0.9629761904761904,
 0.9603571428571429,
 0.963452380952381]

In [16]:
resultados3 = kFold("../data/train.csv",5,10,20,semilla)

In [17]:
resultados3

[0.9602380952380952,
 0.9603571428571429,
 0.9629761904761904,
 0.9603571428571429,
 0.963452380952381]